# Стохастический градиентный и координатный спуски

Для каждого задания указано количество баллов (если они оцениваются отдельно) + 1 балл за аккуратное и полное выполнение всего задания

## Загрузка и подготовка данных

**Загрузите уже знакомый вам файл *Advertising.csv* как объект DataFrame.** 

In [1]:
#ваш код

# импорт необходимых библиотек 
import pandas as pd 
import numpy as np
import plotly
import plotly.express as px

#открываем данные
data = pd.read_csv('Advertising.csv')
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


**Проверьте, есть ли в данных пропуски и, если они есть - удалите их**

In [2]:
#ваш код

# копируем наш датафрейм для дальнейших изменений
advertising_df = data.copy()
# метод isnull() заменяет пропуски на True и не пропуски на False
# Суммируем все пропуски и смотрим на количество пропусков по столбцам
advertising_df.isnull().sum()
# везде нули - пропущенных значений нет 

Unnamed: 0    0
TV            0
radio         0
newspaper     0
sales         0
dtype: int64

**Преобразуйте ваши признаки в массивы NumPy и разделите их на переменные X (предикторы) и y(целевая переменная)** 

In [3]:
#ваш код

# предикторы
TV = np.asarray(advertising_df['TV'])
radio = np.asarray(advertising_df['radio'])
newspaper = np.asarray(advertising_df['newspaper'])
# матрица Х - матрица предикторов
X = np.column_stack((TV, radio, newspaper))
# целевая переменная sales
y = np.asarray(advertising_df['sales'])


## Координатный спуск (3 балла)

**Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:**

In [4]:
import numpy as np
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Нормализуем данные: обычно это необходимо для корректной работы алгоритма**

In [5]:
X = X / np.sqrt(np.sum(np.square(X), axis=0))

**Реализуйте алгоритм координатного спуска:** (3 балла)

Ниже приведен алгоритм:

<a href="https://ibb.co/Th3BQFn"><img src="https://i.ibb.co/DK2DBS6/zascas.jpg" alt="zascas" border="0"></a>

Примечание: 1000 итераций здесь указаны для этого задания, на самом деле их может быть намного больше, нет детерменированного значения.

Вам необходимо реализовать координатный спуск, и вывести веса в модели линейной регрессии.

In [6]:
#ваш код

# w - нулевой вектор весов соответствующей размерности 
w = np.zeros(X.shape[1])

# функция координатного спуска
def coordinate_descent(X, y, w, iter):
# цикл на заданное кол-во итераций
    for iteration in range(iter):
# разница между действиетльным y и предсказанным прогнозом y_pred = (X@w)
        r = y - X@w
# цикл по каждому i-тому значению 
        for i in range(X.shape[1]):
            r_i = (X[:,0:i] @ w[0:i]) + (X[:,i+1:] @ w[i+1:])
# обновление i-того значения весов w
            w[i] = (X[:,i].T @ (y - r_i))
            r -= (X[:,i+1:] @ w[i+1:])
# функция возвращает заполненный вектор весов w
    return w

# применяем функцию на матрице Х, целевой переменной у и векторе весов w, кол-во итераций 1000
print('Вектор весов w =', coordinate_descent(X, y, w,1000))

Вектор весов w = [ 41.56217205 110.13144155  73.52860638  -0.55006384]


Сравните результаты с реализацией линейной регрессии из библиотеки sklearn:

In [7]:
from sklearn.linear_model import LinearRegression
 
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
 
print(model.coef_)

[ 41.56217205 110.13144155  73.52860638  -0.55006384]


Если вы все сделали верно, они должны практически совпасть!

## Стохастический градиентный спуск (6 баллов)

**Отмасштабируйте столбцы исходной матрицы *X* (которую мы не нормализовали еще!). Для того, чтобы это сделать, надо вычесть из каждого значения среднее и разделить на стандартное отклонение** (0.5 баллов)

In [8]:
# ваш код

# изначальная матрица предикторов без дополнительных столбцов
X = np.column_stack((TV, radio, newspaper))
# стандартизация по формуле, описанной в задании
X_std = (X - np.mean(X, axis=0))/ np.std(X, axis=0)
# проверка значений
X_std[:3]

array([[ 0.96985227,  0.98152247,  1.77894547],
       [-1.19737623,  1.08280781,  0.66957876],
       [-1.51615499,  1.52846331,  1.78354865]])

**Добавим единичный столбец**

In [9]:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])

**Создайте функцию mse_error для вычисления среднеквадратичной ошибки, принимающую два аргумента: реальные значения и предсказывающие, и возвращающую значение mse** (0.5 балла)

In [10]:
#ваш код

# аргументы: actual - реальные значения, pred - предсказывающие значения
def mse_error(actual, pred): 
# расчет среднеквадратичной ошибки 
    mse = np.square(actual - pred).mean()
    return mse

**Сделайте наивный прогноз: предскажите продажи средним значением. После этого рассчитайте среднеквадратичную ошибку для этого прогноза** (0.5 балла)

In [11]:
#ваш код

# создаем вектор предсказаний y_pred и заполняем его средним значением
y_pred = np.full(len(y), y.mean())
# расчет mse
mse_error(y,y_pred)

27.085743750000002

**Создайте функцию *lin_pred*, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов** (0.5 балла)

In [12]:
#ваш код

# аргументы: matrix - матрица прогноза, w - веса линейной модели
# функция возвращает вектор прогноза y_pred
def lin_pred(matrix,w):
    y_pred = matrix@w
    return y_pred

**Создайте функцию *stoch_grad_step* для реализации шага стохастического градиентного спуска. (1.5 балла) 
Функция должна принимать на вход следующие аргументы:**
* матрицу *X*
* вектора *y* и *w*
* число *train_ind* - индекс объекта обучающей выборки (строки матрицы *X*), по которому считается изменение весов
* число *$\eta$* (eta) - шаг градиентного спуска

Результатом будет вектор обновленных весов

Шаг для стохастического градиентного спуска выглядит следующим образом:

$$\Large w_j \leftarrow w_j - \frac{2\eta}{\ell} \sum_{i=1}^\ell{{x_{ij}((w_0 + w_1x_{i1} + w_2x_{i2} +  w_3x_{i3}) - y_i)}}$$

Для того, чтобы написать функцию, нужно сделать следующее:
    
*  посчитать направление изменения: умножить объект обучающей выборки на 2 и на разницу между предсказанным значением и реальным, а потом поделить на количество элементов в выборке.
* вернуть разницу между вектором весов и направлением изменения, умноженным на шаг градиентного спуска

In [13]:
#ваш код

# аргументы: матрица X, вектора y и w, train_ind - индекс объекта обуч. выборки, eta - шаг градиентного спуска
def stoch_grad_step(X,y,w,train_ind,eta):
# обозначение строк матрицы Х и вектора у
    x_i = X[train_ind]  
    y_i = y[train_ind] 
# расчет направления
    y_pred = lin_pred(x_i.reshape(1, -1), w)
    direction = (x_i*2*(y_pred - y_i))/len(X)
# расчет нового вектора  
    w_new = w - eta * direction
# возвращается вектор обновленных весов
    return w_new

**Создайте функцию *stochastic_gradient_descent*, для реализации стохастического градиентного спуска (2.5 балла)**

**Функция принимает на вход следующие аргументы:**
- Матрицу признаков X
- Целевую переменнную
- Изначальную точку (веса модели)
- Параметр, определяющий темп обучения
- Максимальное число итераций
- Евклидово расстояние между векторами весов на соседних итерациях градиентного спуска,при котором алгоритм прекращает работу 

**На каждой итерации в вектор (список) должно записываться текущее значение среднеквадратичной ошибки. Функция должна возвращать вектор весов $w$, а также вектор (список) ошибок.**

Алгоритм сследующий:
    
* Инициализируйте расстояние между векторами весов на соседних итерациях большим числом (можно бесконечностью)
* Создайте пустой список для фиксации ошибок
* Создайте счетчик итераций
* Реализуйте основной цикл обучения пока расстояние между векторами весов больше того, при котором надо прекратить работу (когда расстояния станут слишком маленькими - значит, мы застряли в одном месте) и количество итераций меньше максимально разрешенного: сгенерируйте случайный индекс, запишите текущую ошибку в вектор ошибок, запишите в переменную текущий шаг стохастического спуска с использованием функции, написанной ранее. Далее рассчитайте текущее расстояние между векторами весов и прибавьте к счетчику итераций 1.
* Верните вектор весов и вектор ошибок

In [14]:
# ваш код

# аргументы: Матрица X, у - целевая переменная, weights - изначальная точка (веса модели)
# rate - параметр темпа обучения, max_iter - max число итераций, evklid - минимальное расстояние между векторами
def stochastic_gradient_descent(X,y,weights,rate,max_iter,evklid):
# пустой список для записи mse на каждой итерации
    mse_list = list()
# расстояние между векторами весов на соседних итерациях - бесконечность
    weight_dist = np.inf 
# создаем счетчик итераций
    iter_count = 0
# цикл обучения
    while iter_count < max_iter and weight_dist > evklid:
# генерация случайного индекса из матрицы X
        train_ind = np.random.randint(X.shape[0])
# получение новых весов на основе шага 
        new_weights = stoch_grad_step(X, y, weights, train_ind, rate)
# расчет расстояния между весами     
        weight_dist = np.linalg.norm(new_weights - weights)
# перезапись новых весов для реализации последующих итераций    
        weights = new_weights
# расчет прогноза       
        y_pred = lin_pred(X, weights)  
# расчет ошибки и добавление в список ошибок
        error = mse_error(y, y_pred)
        mse_list.append(error)
# работа счетчика
        iter_count += 1
    
# возвращается вектор w и векор ошибок
    return weights, mse_list

 **Запустите $10^5$ итераций стохастического градиентного спуска. Укажите вектор начальных весов, состоящий из нулей. Можете поэкспериментировать с параметром, отвечающим за темп обучения.**

**Постройте график зависимости ошибки от номера итерации**

In [15]:
# ваш код

# начальный вектор весов соответствующий размерности матрице Х
w_start = np.zeros(X.shape[1])
# задаем случайный темп обучения
rate = 0.00005
# кол-во итераций
iter = 10**5
# минимальное расстояние между векторами на соседних итерациях
evklid = 10**(-8)
# реализация алгоритма спуска
w, mse_list = stochastic_gradient_descent(X,y,w_start,rate,iter,evklid)

# создаем датафрейм для построения графика с номером итерации и соответствующей ошибкой
iteration = range(1,len(mse_list)+1)
df = pd.DataFrame(mse_list)
df.insert(loc=0, column='iteration', value=iteration)
df.columns = ['iteration', 'MSE']

In [16]:
# график зависимости ошибки от номера итерации
fig = px.line(
    data_frame=df,
    x='iteration', 
    y= 'MSE',
    height=500,
    width=500,
    title='График зависимости ошибки от номера итерации' 
)
fig.show()

**Выведите вектор весов, к которому сошелся метод.**

In [17]:
# ваш код

print('Вектор весов w:', w)

Вектор весов w: [0.0098019  0.05244569 0.21560347 0.02102737]


**Выведите среднеквадратичную ошибку на последней итерации.**

In [18]:
# ваш код

print('MSE на последней итерации:', mse_list[-1])

MSE на последней итерации: 4.1073913081601106


### БОНУСНОЕ ЗАДАНИЕ

Разумеется, в реальности специалисты по машинному обучению не прописывают алгоритмы с нуля, а пользуются готовыми реализациями из библиотек. Реализованный Вами стохастический градиентный спуск представлен в библиотеке sklearn.

Что необходимо сделать:

Выберите в документации класс, подходящий для вашей задачи, и обучите стохастический градиентный спуск.

Оцените качество с помощью среднеквадратичной ошибки и сравните результат с результатом алгоритма, реализованного вами самостоятельно.

В одном из предыдущих модулей качество решения задачи регрессии оценивалось с помощью ещё одной метрики. Найдите её реализацию в библиотеке sklearn и оцените качество полученной модели.

In [19]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [20]:
# изначальная матрица предикторов без дополнительных столбцов
X = np.column_stack((TV, radio, newspaper))
# стандартизация и добавление единичного столбца (интерсепт)
X_std = (X - np.mean(X, axis=0))/ np.std(X, axis=0)
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
# y - целевая переменная продаж 
y = np.asarray(advertising_df['sales'])
# для оценки качества, разделяем данные на тестовую и тренировочную выборки 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# применяем SGDRegressor для реализации стохастического градиентного спуска
discent = SGDRegressor(loss='squared_error', 
                       penalty='elasticnet', 
                       alpha=0.01, 
                       l1_ratio=0.05, 
                       fit_intercept=True, 
                       max_iter=100000, 
                       tol=0.00000001,
                       eta0=0.00005)
'''
Аргументы:
loss - расчет потерь 
penalty - параметр штрафа
alpha - параметр для регуляризации
l1_ratio - параметр для регуляризации
fit_intercept - вставка интерсепта
max_iter - макс. кол-во итераций задаем как в предлагалось по условиям задачи 10**5
tol - критерий остановки равный выбранному евклидову расстоянию - 10**(-8)
eta0 - начальный уровень обучения, как и в написанной функции - rate=0.00005
'''
# обучаем алгоритм на тренировочной выборке
# Обучаем scaler на обучающем наборе и преобразуем обучающий набор
discent.fit(X_train_scaled, y_train)
# полученный вектор весов
new_w = discent.coef_
# расчет прогноза на тестовой выборке
y_pred2 = X_test@new_w

In [22]:
# Реализуем написанную вручную функцию с теми же аргументами, но на тренировочной выборке
w_start = np.zeros(X.shape[1])
w, mse_list = stochastic_gradient_descent(X_train,y_train,w_start,rate=0.00005,max_iter=100000,evklid=10**(-8))
# расчет прогноза на тестовой выборке
y_pred1 = X_test@w

In [23]:
# расчет абсолютной ошибки
absolute = abs(y_pred1 - y_pred2)
# расчет среднеквадратичной ошибки
mse = mse_error(y_pred1,y_pred2)

print('абсолютная ошибка:',round(float(absolute[0])))
print('MSE =',round(mse,2))

абсолютная ошибка: 0
MSE = 0.16


Таким образом, при заданных одинаковых параметрах обе (написанная нами и втроенная функция из библиотеки Sklearn) дают одинаковый результат (абсолютная ошибка = 0).